Set up tables

In [1]:
import duckdb
conn = duckdb.connect('mydb.duckdb')

# Drop table if exists
conn.sql("DROP TABLE IF EXISTS comments")
conn.sql("DROP TABLE IF EXISTS videos")

# Create 'comments' table from multiple CSVs
conn.sql("""
    CREATE TABLE IF NOT EXISTS comments AS
    SELECT * FROM read_csv_auto('dataset/comments1.csv')
    UNION ALL
    SELECT * FROM read_csv_auto('dataset/comments2.csv')
    UNION ALL
    SELECT * FROM read_csv_auto('dataset/comments3.csv')
    UNION ALL
    SELECT * FROM read_csv_auto('dataset/comments4.csv')
    UNION ALL
    SELECT * FROM read_csv_auto('dataset/comments5.csv')
""")

# Add relevance score column
conn.sql("""
    ALTER TABLE comments ADD COLUMN relevance_score DOUBLE;
""")

# Create 'videos' table from videos.csv
conn.sql("""
    CREATE TABLE IF NOT EXISTS videos AS
    SELECT * FROM read_csv_auto('dataset/videos.csv')
""")

Show table columns

In [3]:
# Show all columns for 'comments' and 'videos' tables
print('Comments table columns:')
print(conn.sql('PRAGMA table_info(comments)').df()['name'].tolist())

print('Videos table columns:')
print(conn.sql('PRAGMA table_info(videos)').df()['name'].tolist())

Comments table columns:
['kind', 'commentId', 'channelId', 'videoId', 'authorId', 'textOriginal', 'parentCommentId', 'likeCount', 'publishedAt', 'updatedAt', 'relevance_score']
Videos table columns:
['kind', 'videoId', 'publishedAt', 'channelId', 'title', 'description', 'tags', 'defaultLanguage', 'defaultAudioLanguage', 'contentDuration', 'viewCount', 'likeCount', 'favouriteCount', 'commentCount', 'topicCategories']


Calculate RELEVANCE

In [ ]:
import pandas as pd

# TEST: Calculate a relevance score
# relevance score = (likeCount * 2) + (lengthOfComment * 0.01) - (ageInDays * 0.1)
conn.sql("""
    UPDATE comments SET relevance_score =
        COALESCE(likeCount, 0) * 2
        + COALESCE(LENGTH(textOriginal), 0) * 0.01
        + COALESCE(julian(CAST(now() AS TIMESTAMP)) - julian(CAST(publishedAt AS TIMESTAMP)), 0) * -0.1
""")

# View top relevant comments by the new score
top_comments = conn.sql('SELECT commentId, textOriginal, likeCount, relevance_score FROM comments ORDER BY relevance_score DESC LIMIT 10').df()
pd.set_option('display.max_colwidth', None)
display(top_comments)

,commentId,textOriginal,likeCount,relevance_score
0,2157929,men will say they don’t like makeup and then call a women wearing a full face the hottest person ever.,456142,912209.470535
1,3226321,Unpopular opinion… we aren’t colors we’re people 💪,361860,723693.641991
2,3626502,They really be discovering now that black people can tan too 😂,236006,471927.330128
3,850913,The fade into the makeup filter is flawless.. I’m gonna try the tape,210116,420147.012792
4,899447,Oh thank God. For a few seconds there you were Marge Simpson.,194610,389164.050780
5,2153599,"The homie nailed when he said"" we are all different shades of wheat""",162767,325511.577481
6,4623242,if this man is struggling to get asked out we're all fucked,143374,286671.274188
7,3711365,I NEVER KNEW SOMEONE ALSO STRUGGLES WITH THE PERFECT PART,139006,277960.095769
8,4421158,Dude for people trying to say her hair isn’t curly I could personally use every product she uses and more and do it the same way and my hair will not be curly I have straight hair she doesn’t lol she spoke straight facts,138535,277002.861440
9,2387235,you know she’s pretty when she can pull off everything 💕\n\n\ny’all be blowing up my phone 😭💀,137338,274596.418533


In [12]:
conn.sql('SHOW ALL TABLES').df()

,database,schema,name,column_names,column_types,temporary
0,mydb,main,comments,"[kind, commentId, channelId, videoId, authorId, textOriginal, parentCommentId, likeCount, publishedAt, updatedAt, relevance_score]","[VARCHAR, BIGINT, BIGINT, BIGINT, BIGINT, VARCHAR, BIGINT, BIGINT, TIMESTAMP WITH TIME ZONE, TIMESTAMP WITH TIME ZONE, DOUBLE]",False
1,mydb,main,videos,"[kind, videoId, publishedAt, channelId, title, description, tags, defaultLanguage, defaultAudioLanguage, contentDuration, viewCount, likeCount, favouriteCount, commentCount, topicCategories]","[VARCHAR, BIGINT, TIMESTAMP WITH TIME ZONE, BIGINT, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, VARCHAR, DOUBLE, DOUBLE, DOUBLE, DOUBLE, VARCHAR]",False


In [ ]:
conn.sql('select * from comments limit 100').df()


In [ ]:
conn.sql('drop table comments')

In [ ]:
conn.sql('describe comments').df()

In [ ]:
conn.sql('select count(*) from comments').df()

In [ ]:
conn.sql("select * from comments where videoId='1265'").df()

In [ ]:

conn.sql('select * from comments').df()

In [ ]:
# Explore the videos table
conn.sql('select * from videos limit 100').df()



In [ ]:
conn.sql('describe videos').df()

In [ ]:
conn.sql('select * from videos').df()

In [ ]:
video_df = conn.sql('select * from videos').df()
comment_df = conn.sql('select * from comments').df()

In [ ]:
# Join comments with videos on videoId
joined_df = conn.sql('''
SELECT c.*, v.*
FROM comments c
JOIN videos v ON c.videoId = v.videoId
''').df()

In [ ]:
# Display the joined DataFrame
pd.set_option('display.max_rows', None)      # Show all rows
pd.set_option('display.max_columns', None)   # Show all columns
# show all rows and columns
joined_df.head(1000)

In [ ]:
# Drop unnecessary columns
cleaned_df = joined_df.drop(columns=['kind', 'kind_1', 'videoId_1', 'publishedAt_1', 'channelId_1', 'likeCount_1'])

In [ ]:
cleaned_df.head(100)

In [ ]:
# count the number of null values in contentDuration column
for column in cleaned_df.columns:
    num_nulls = cleaned_df[column].isnull().sum()
    print(f"The number of null values in {column} column is: {num_nulls}")


In [ ]:
# Convert the contentDuration column to total seconds from ISO 8601 format
cleaned_df.shape